# Quick Hershey font example

By [Allison Parrish](http://www.decontextualize.com/)

This notebook shows how to render Hershey fonts with [Flat](https://xxyxyz.org/flat) and [Bezmerizing](https://github.com/aparrish/bezmerizing/). I'm going to use this [JSON version of the Hershey fonts](https://github.com/techninja/hersheytextjs), included in this repository as [hersheytext.json](hersheytext.json).

## Getting path data for a character

First, we'll load the JSON:

In [6]:
import json

In [7]:
all_data = json.load(open("./hersheytext.json"))

The JSON file consists of a dictionary whose keys are individual Hershey fonts. The character data for these fonts is stored as the value for the key `chars` in a nested dictionary. Here's a list of all available fonts:

In [86]:
all_data.keys()

dict_keys(['astrology', 'cursive', 'cyrillic', 'futural', 'futuram', 'gothiceng', 'gothicger', 'gothicita', 'greek', 'japanese', 'markers', 'mathlow', 'mathupp', 'meteorology', 'music', 'scriptc', 'scripts', 'symbolic', 'timesg', 'timesi', 'timesib', 'timesr', 'timesrb'])

The cell below grabs the sans-serif 1-stroke Hershey font:

In [8]:
futural = all_data['futural']['chars']

The resulting value is a list that has path data for each letter. To find the index for a letter, take its ASCII value and subtract 33:

In [9]:
def idx(ch):
    return ord(ch) - 33

In [10]:
futural[idx('g')]

{'d': 'M15,8 L15,24 14,27 13,28 11,29 8,29 6,28 M15,11 L13,9 11,8 8,8 6,9 4,11 3,14 3,16 4,19 6,21 8,22 11,22 13,21 15,19',
 'o': 10}

This returns *yet another dictionary*; the path data for the character is the value for key `d`. The function below combines all of this together, giving you the path data for the character in the font that you specify:

In [11]:
def dch(font, ch):
    if ch == ' ':
        return 'M8,0'
    else:
        return font[idx(ch)]['d']
print(dch(futural, 'a'))
print(dch(futural, ' '))

M15,8 L15,22 M15,11 L13,9 11,8 8,8 6,9 4,11 3,14 3,16 4,19 6,21 8,22 11,22 13,21 15,19
M8,0


## Interpreting paths

The path data is stored as [SVG path commands](https://developer.mozilla.org/en-US/docs/Web/SVG/Tutorial/Paths). Only two commands are used: `M` (move to a coordinate) and `L` (draw a line from the current coordinate to the given coordinate, moving the cursor to that coordinate). The following function parses the command string into a Bezmerizing `Path` object, which we can then draw with Flat.

> Note that this is *not* a general function for parsing SVG paths—it only works for the subset of commands that happen to be used in this Hershey font JSON file. The [svg.path package](https://github.com/regebro/svg.path) has an actual SVG path parser.

In [14]:
from flat import command
from bezmerizing import Path

In [15]:
import numpy as np
def topath(font, ch):
    cmds = []
    mode = ''
    for t in dch(font, ch).split():
        if t[0] in ('M', 'L'):
            mode = t[0]
            t = t[1:]
        coords = [int(x) for x in t.split(',')]
        if mode == 'M':
            cmds.append(command.moveto(*coords))
        else:
            cmds.append(command.lineto(*coords))
    return Path(cmds)

Now the `topath()` function takes a Hershey font object (retrieved from the JSON) and the character you want to convert to a path:

In [16]:
topath(futural, "x")

Path([moveto(x=3.0000, y=8.0000), lineto(x=14.0000, y=22.0000), moveto(x=14.0000, y=8.0000), lineto(x=3.0000, y=22.0000)])

Let's draw this real quick with Flat. First, some Flat code that we need to draw and display:

In [19]:
from flat import document, shape, rgb
from IPython.display import SVG, display
def show(page):
    display(SVG(page.svg()))

And now actually draw a character:

In [22]:
page = document(20, 20, 'mm').addpage()
pen = shape().stroke(rgb(0, 0, 0)).width(2)
page.place(pen.path(topath(futural, '&')))
show(page)

Very cool. Because this is a `Path` object, we can perform transformations on it:

In [26]:
amp_path = topath(futural, '&')
page = document(30, 20, 'mm').addpage()
pen = shape().stroke(rgb(0, 0, 0)).width(2)
page.place(pen.path(amp_path.scalexy(1.5, 0.75)))
show(page)

## Laying out text with Hershey fonts

The following function lays out a string with the specified Hershey font on a single line, by creating a new composite path that contains each Hershey character, offset by the width of the character that came before. The `kern` parameter controls how much spacing is inserted between each character.

In [27]:
from itertools import chain
def strtopath(font, s, kern=1):
    current_x = 0
    all_paths = []
    for ch in s:
        path = topath(font, ch).translate(current_x, 0)
        all_paths.append(path)
        x_extent = max([c.x for c in path.commands])
        current_x = x_extent + kern
    return Path(list(chain(*all_paths)))

Here's what it looks like:

In [31]:
str_path = strtopath(futural, "Quartzy foxes").scale(0.5)
page = document(100, 16, 'mm').addpage()
pen = shape().stroke(rgb(0, 0, 0)).width(2)
page.place(pen.path(str_path))
show(page)

The following code incorporates word wrapping to lay out text on multiple lines. It uses the `strtopath` method to generate paths for each word, then checks to see if the width of the word would extend past the edge of the box (given the current cursor position). If so, it puts that word on the next line instead.

In [39]:
def strtopath_wrap(font, s, kern=1, lineheight=32, width=360, height=360):
    current = [0, 0]
    all_paths = []
    for word in s.split():
        wpath = strtopath(font, word, kern)
        wpath_width = max([c.x for c in wpath.commands])
        if current[0] + wpath_width > width:
            current[1] += lineheight
            current[0] = 0
        if current[1] > height - lineheight:
            break
        all_paths.append(wpath.translate(current[0], current[1]))
        current[0] += wpath_width + 8
    return Path(list(chain(*all_paths)))

Here's a quote from Johanna Drucker's *The Visible Word*, laid out with a Hershey font:

In [46]:
src = '''\
The threat to linguistic authority made by the manipulation
of the words on the page was that it returned the written
language to the specific place, instance, conditions of
production: it became a highly marked text. The unmarked
text... appeared, as it were, to "speak itself."'''

In [47]:
doc_height = 360 # points
doc_width = 360 # points
d = document(doc_width, doc_height, 'pt')
page = d.addpage()
line_fig = shape().stroke(rgb(0, 0, 0)).width(2)
text_path = strtopath_wrap(futural, src).translate(24, 24)
page.place(line_fig.path(text_path))
show(page)

## Modifying coordinates with simplex noise

[Simplex noise](https://en.wikipedia.org/wiki/Simplex_noise) is a generalized and improved version of [Perlin noise](https://en.wikipedia.org/wiki/Perlin_noise). The Python [opensimplex](https://github.com/lmas/opensimplex) library provides an efficient and user-friendly interface for producing Simplex noise. We're going to use it to do weird stuff to the coordinates in our Hershey text. First, install the library:

In [48]:
import sys
!{sys.executable} -m pip install opensimplex

You should consider upgrading via the 'pip install --upgrade pip' command.


Then import it, and create an `OpenSimplex` object:

In [49]:
import opensimplex

In [50]:
noisy = opensimplex.OpenSimplex()

An `OpenSimplex` object has a couple of methods that we'll use. The `.noise2d()` method takes two parameters (i.e., a 2d coordinate) and returns a random number for that coordinate:

In [51]:
noisy.noise2d(10, 20)

-0.465547161728568

The random values range from -1 to 1. Simplex noise has the property that random numbers generated from nearby points will also be similar:

In [54]:
noisy.noise2d(10.01, 20.01)

-0.47270926394432106

The `.noise3d()` method does the same thing, but with 3d coordinates instead of 2d coordinates:

In [58]:
noisy.noise3d(1, 5, 9)

-0.029126213592233115

As a quick example, let's draw a 2d grid of ellipses whose fill color is determined by the simplex noise at the ellipse's coordinate:

In [84]:
width = 200
height = 200
grid_size = 12
cell_size = width/grid_size
noise_step = 0.02
page = document(width, height, 'pt').addpage()
for i in range(grid_size):
    for j in range(grid_size):
        x = (i * cell_size) + cell_size * 0.5
        y = (j * cell_size) + cell_size * 0.5
        val = noisy.noise2d(x*noise_step, y*noise_step)
        val_scaled = (val * 128) + 128
        pen = shape().nostroke().fill(rgb(val_scaled, val_scaled, val_scaled))
        page.place(pen.ellipse(x, y, cell_size*0.5, cell_size*0.5))
show(page)

Try adjusting the value of `noise_step`. If the value is too large, then you end up sampling from the noise at points that are too distant from each other, so the resulting values are very different from each other. At around 0.01 or 0.02, you see that the colors of adjacent circles are random, but still similar to those that surround them.

I'll use this same technique to make our Hershey text look "warped" by adding values generated by simplex noise to the `x` and `y` coordinates of each command in the Hershey text path. (Note that I'm using `noise3d` here as a convenience. I need to generate two values for each coordinate—an x-offset and an y-offset—so I use the `z` coordinate of `.noise3d()` in order to sample values from a different part of the simplex noise "space" for each `x, y` coordinate.) Just for fun, I picked a different font (`scriptc`) instead of the `futural` font used above.

In [95]:
text_path = strtopath_wrap(all_data['scriptc']['chars'], src, kern=0)
doc_height = 240 # points
doc_width = 240 # points
d = document(doc_width, doc_height, 'mm')
page = d.addpage()
line_fig = shape().stroke(rgb(0, 0, 0)).width(3).join('round').cap('round')
for i in range(1):
    manip = []
    for cmd in text_path:
        nx = noisy.noise3d(cmd.x*0.02, cmd.y*0.02, 100*i*0.02) * 8
        ny = noisy.noise3d(cmd.x*0.02, cmd.y*0.02, 100+(100*i*0.02)) * 8
        cmd.x += nx
        cmd.y += ny
    manip_tr = text_path.scale(0.75).translate(24, 24)
    this_text = line_fig.path(manip_tr)
    page.place(this_text)
show(page)